# Agentic Retrieval Quickstart for Azure AI Search

1. Create index
1. Ingest sample data
1. Create agent
1. Run agentic retrieval
1. Generate response using chat completion
1. Generate repsonse using responses